**LSTM stock price prediction**

In [2]:
#install required libraries 
!pip install yfinance
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 KB 979.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.3/114.3 KB 4.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19790 sha256=26c71ce6edc301086f56d4fdb86fe83061b2ddb6b7008a765bd8738dcd19b23d
  Stored in directory: /root/.cache/pip/wheels/f6/89/59/49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


In [3]:

import math
import json
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, LSTM
import datetime as dt
import getpass
from pyngrok import ngrok, conf
import os
import threading
from flask import Flask

# list of stocks to predict
stocks = ['AAPL', 'AMZN', 'GOOG', 'MSFT', 'META', 'TSLA', 'BRK-B', 'DIS','BTC-USD', 'ETH-USD']

# dictionary to store prediction values
predictions = {}


**Data Preprocessing**

In [4]:
# loop for getting predition of each stock

for stock in stocks:
    # download stock data from Yahoo Finance API
    stock_data = yf.download(stock, start='2020-01-01', end='2023-03-01')
    
    # extract closing prices
    closing_Arr = stock_data['Close']
    closing_Price = closing_Arr.values

    # Scale the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_prices = scaler.fit_transform(closing_Price.reshape(-1,1))

    # Create sequences of length 60
    seq_length = 60
    X = []
    y = []
    for i in range(seq_length, len(scaled_prices)):
        X.append(scaled_prices[i-seq_length:i, 0])
        y.append(scaled_prices[i, 0])

    X = np.array(X)
    y = np.array(y)

    # Reshape the input data for LSTM
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))
    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(units=100, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(LSTM(units=100))
    model.add(Dense(units=25))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Train the model
    model.fit(X, y, epochs=1, batch_size=32)

    # Use the model to make predictions for the next 14 days
    next_days = []
    for i in range(14):
        last_seq = closing_Price.reshape(-1,1)[-seq_length:]
        last_seq_scaled = scaler.transform(last_seq)
        next_day_scaled = model.predict(np.array([last_seq_scaled]))
        next_day = scaler.inverse_transform(next_day_scaled)[0][0]
        next_days.append(next_day)
        closing_Price = np.append(closing_Price.reshape(-1,1), [[next_day]], axis=0)

    dates_stock = closing_Arr.index
    last_date = dates_stock[-1]

    # get the last date in the index
    last_date = closing_Arr.index[-1]

    # create an array of the next 14 days
    date_range = pd.date_range(last_date, periods=14, freq='D')

    # convert the array to a numpy array
    date_range_array = np.array(date_range)

    Dates = np.concatenate([dates_stock, date_range_array])
    # Convert the dates to strings in the format 'YYYY-MM-DD'
    Dates = Dates.astype('datetime64[D]').astype(str)

    # Add the predicted closing prices and dates to the dictionary
    predictions[stock] = {'Dates': Dates, 'close': closing_Price.tolist()} 
    print(stock)


[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 38ms/step
AAPL
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 36ms/step
AMZN
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 35ms/step
GOOG
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 38ms/step
MSFT
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 42ms/step
META
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 36ms/step
TSLA
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 33ms/step
BRK-B
[*********************100%***********************]  1 of 1 completed
1/1 [==============================] - 0s 33ms/s

**pyngrok**

In [5]:
#pyngrok to open a tunnel to that server.
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open a TCP ngrok tunnel to the SSH server
connection_string = ngrok.connect(22, "tcp").public_url

ssh_url, port = connection_string.strip("tcp://").split(":")
print(f" * ngrok tunnel available, access with `ssh root@{ssh_url} -p{port}`")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········
 * ngrok tunnel available, access with `ssh root@0.tcp.ngrok.io -p19019`


**Flask**

In [ ]:

os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)
port = 5000

# Open a ngrok tunnel to the HTTP server
public_url = ngrok.connect(port).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# ... Update inbound traffic via APIs to use the public-facing ngrok URL


# Define Flask routes
@app.route("/")
def index():
    prediction_list = []
    for stock, data in predictions.items():
        dates = data['Dates']
        prices = data['close']
        for i in range(len(dates)):
            prediction = {}
            prediction['date'] = dates[i]
            prediction['closing_price'] = prices[i]
            prediction['stock'] = stock
            prediction_list.append(prediction)

    # Convert the list of dictionaries to JSON format
    json_predictions = json.dumps(prediction_list, default=str, indent=4)
    
    return json_predictions


# Start the Flask server in a new thread
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()

In [35]:
# disconnect and kill the ngrok and flask
ngrok.disconnect(public_url)
ngrok.kill()